# Using Dataclasses

Heavylight doesn't prescribe any specific input structures, so you can store data directly in attributes, or as classes, dictionaries (or even directly query databases for data).  Using `dataclass` allows for type checking and autocompletion.

In [51]:
from dataclasses import dataclass, fields
from heavylight import Model, Table
from typing import Literal

In [60]:
# Rather than dictionaries etc, we can also specify model data/assumptions etc as dataclasses.
# (caveat - dataclasses use dictionaries anyway.)

@dataclass
class Data:
    init_age: int
    init_fund: float
    term_m: int

@dataclass
class Basis:
    growth_rate_pm: float
    q_x: Table


In [40]:
class TestModel(Model):
    # specify the data and basis types so we can typecheck/autocomplete.
    data: Data
    basis: Basis
    product: Literal['ProdA' , 'ProdB']
    
    def t(self, t):
        return t
    
    def age(self, t):
        if t == 0:
            return self.data.init_age
        elif t % 12 == 0:
            return self.age(t - 1) + 1
        else:
            return self.age(t - 1)
    
    def fund_value(self, t):
        if t == 0:
            return self.data.init_fund
        else:
            return self.fund_value(t - 1) * (1 + self.basis.growth_rate_pm)

    def product_ind(self, t):
        if self.product == 'ProdA':
            return 1
        elif self.product == 'ProdB':
            return 2
        else:
            return -1

    def q_x_m(self, t):
        return self.basis.q_x[t] / 12

    def months_to_maturity(self, t):
        return max(self.data.term_m - t, 0)

In [41]:
# note that positional assignment can be used, but care needs to be taken that inputs aren't transposed
data = Data(35, 1234.5, 14)
basis = Basis(growth_rate_pm = 0.04/12, q_x = Table.read_csv('sample_q_x_table.csv'))

In [46]:
model = TestModel(data=data, basis=basis, proj_len=20, product='ProdA')

In [45]:
model.df

,t,age,fund_value,months_to_maturity,product_ind,q_x_m
0,0,35,1234.500000,14,-1,0.005510
1,1,35,1238.615000,13,-1,0.005721
2,2,35,1242.743717,12,-1,0.005941
3,3,35,1246.886196,11,-1,0.006170
4,4,35,1251.042483,10,-1,0.006407
5,5,35,1255.212625,9,-1,0.006654
6,6,35,1259.396667,8,-1,0.006911
7,7,35,1263.594656,7,-1,0.007178
8,8,35,1267.806638,6,-1,0.007456
9,9,35,1272.032660,5,-1,0.007745


In [34]:
model.basis.growth_rate_pm

0.0033333333333333335

In [35]:
model.data.init_age

35

In [36]:
model.q_x_m.df

,t,q_x_m
0,0,0.005510
1,1,0.005721
2,2,0.005941
3,3,0.006170
4,4,0.006407
5,5,0.006654
6,6,0.006911
7,7,0.007178
8,8,0.007456
9,9,0.007745


In [65]:
TestModel(proj_len=5, data=data, basis=basis, product='ProdB').df

,t,age,fund_value,months_to_maturity,product_ind,q_x_m
0,0,35,1234.500000,14,2,0.005510
1,1,35,1238.615000,13,2,0.005721
2,2,35,1242.743717,12,2,0.005941
3,3,35,1246.886196,11,2,0.006170
4,4,35,1251.042483,10,2,0.006407


In [54]:
fields(data)

(Field(name='init_age',type=<class 'int'>,default=<dataclasses._MISSING_TYPE object at 0x1017db650>,default_factory=<dataclasses._MISSING_TYPE object at 0x1017db650>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 Field(name='init_fund',type=<class 'float'>,default=<dataclasses._MISSING_TYPE object at 0x1017db650>,default_factory=<dataclasses._MISSING_TYPE object at 0x1017db650>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 Field(name='term_m',type=<class 'int'>,default=<dataclasses._MISSING_TYPE object at 0x1017db650>,default_factory=<dataclasses._MISSING_TYPE object at 0x1017db650>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD))

In [55]:
# walking from one basis to another